# Error Handling and Resilience

This notebook demonstrates robust error handling, retry strategies, fallback mechanisms, and resilience patterns in lionpride. Learn how to build production-ready applications that gracefully handle failures.

## What You'll Learn

1. **Common errors** - Rate limits, timeouts, validation failures
2. **Retry patterns** - Exponential backoff and intelligent retries
3. **Circuit breakers** - Prevent cascading failures
4. **Fallback chains** - Graceful degradation across providers
5. **Validation recovery** - Handle and correct structured output errors
6. **Timeout handling** - Set reasonable time boundaries
7. **Production patterns** - Best practices for resilient systems

## Setup

First, let's import the required libraries:

In [ ]:
import asyncio
from datetime import datetime

from pydantic import BaseModel, Field, ValidationError

from lionpride import Session

# Use lionpride's built-in concurrency primitives
from lionpride.libs.concurrency import fail_after, retry
from lionpride.operations import communicate
from lionpride.services import iModel

## 1. Basic Error Handling

Understanding common errors and how to handle them gracefully.

In [ ]:
async def basic_error_handling():
    """Handle common errors in LLM operations"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    branch = session.create_branch(name="error-demo")

    try:
        result = await communicate(
            session=session,
            branch=branch,
            parameters={
                "instruction": "Write a short poem about coding",
                "imodel": model.name,
            },
        )
        print(f"✓ Success: {result[:100]}...")

    except ValueError as e:
        print(f"✗ Configuration error: {e}")
        # Invalid parameters, missing required fields

    except RuntimeError as e:
        print(f"✗ Execution error: {e}")
        # API call failed, timeout, rate limit

    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        # Catch-all for other errors


# Run the example
await basic_error_handling()

### Error Hierarchy

```
Exception
├── ValueError          # Configuration errors (invalid parameters)
├── RuntimeError        # Execution errors (API failures, timeouts)
├── ValidationError     # Pydantic validation failures
└── TimeoutError        # Operation timeouts
```

**Key principle**: Handle specific exceptions before general ones.

## 2. Retry with Exponential Backoff

Transient failures (network issues, temporary rate limits) can be resolved by retrying with increasing delays.

In [ ]:
async def communicate_with_retry(session, branch, parameters):
    """Retry operation with exponential backoff using lionpride's retry"""
    return await retry(
        lambda: communicate(session=session, branch=branch, parameters=parameters),
        attempts=3,
        base_delay=1.0,
        max_delay=10.0,
        retry_on=(RuntimeError,),  # Only retry runtime errors (API failures, timeouts)
    )


async def retry_pattern():
    """Use retry logic for transient failures"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    branch = session.create_branch(name="retry")

    try:
        result = await communicate_with_retry(
            session=session,
            branch=branch,
            parameters={
                "instruction": "Explain retry patterns in software engineering",
                "imodel": model.name,
            },
        )
        print(f"✓ Success after retries: {result[:150]}...")

    except Exception as e:
        print(f"✗ Failed after all retries: {e}")


# Run the example
await retry_pattern()

### Retry Configuration (lionpride.libs.concurrency.retry)

- **attempts=3**: Maximum 3 attempts
- **base_delay=1.0**: Start with 1 second delay
- **max_delay=10.0**: Cap at 10 seconds
- **retry_on=(RuntimeError,)**: Only retry RuntimeErrors (not config errors)
- **jitter=0.1**: Small random variation to prevent thundering herd

**Decision tree**:
```
Error occurs
├── Transient? (network, rate limit, timeout) → Retry with backoff
├── Configuration? (invalid params) → Fix and retry
├── Validation? (bad output) → Retry with correction
└── Permanent? (auth, quota) → Fail fast
```

## 3. Circuit Breaker Pattern

Prevent cascading failures by "opening" the circuit after repeated failures.

In [ ]:
class CircuitBreaker:
    """Circuit breaker to prevent cascading failures"""

    def __init__(self, failure_threshold=3, timeout=60):
        self.failure_threshold = failure_threshold
        self.timeout = timeout  # seconds
        self.failures = 0
        self.last_failure_time = None
        self.state = "closed"  # closed, open, half-open

    def call(self, func):
        """Wrap function with circuit breaker logic"""

        async def wrapper(*args, **kwargs):
            if self.state == "open":
                # Check if timeout has passed
                if (datetime.now() - self.last_failure_time).seconds > self.timeout:
                    print("⚠️  Circuit breaker: Transitioning to HALF-OPEN")
                    self.state = "half-open"
                else:
                    raise RuntimeError("Circuit breaker is OPEN - service unavailable")

            try:
                result = await func(*args, **kwargs)

                # Success - reset if half-open
                if self.state == "half-open":
                    print("✓ Circuit breaker: Transitioning to CLOSED")
                    self.state = "closed"
                    self.failures = 0

                return result

            except Exception:
                self.failures += 1
                self.last_failure_time = datetime.now()

                if self.failures >= self.failure_threshold:
                    print(f"✗ Circuit breaker: OPENING after {self.failures} failures")
                    self.state = "open"

                raise

        return wrapper


async def circuit_breaker_demo():
    """Use circuit breaker to protect against repeated failures"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    breaker = CircuitBreaker(failure_threshold=3, timeout=5)

    @breaker.call
    async def protected_communicate(instruction):
        branch = session.create_branch(name=f"attempt-{datetime.now().timestamp()}")
        return await communicate(
            session=session,
            branch=branch,
            parameters={
                "instruction": instruction,
                "imodel": model.name,
            },
        )

    # Test with valid calls (should succeed)
    print("Testing circuit breaker with valid calls:\n")

    for i in range(2):
        try:
            result = await protected_communicate(f"What is {i + 1} + {i + 1}?")
            print(f"Attempt {i + 1}: ✓ Success - {result}")
        except Exception as e:
            print(f"Attempt {i + 1}: ✗ {e}")
        await asyncio.sleep(0.5)

    print(f"\nCircuit breaker state: {breaker.state}")
    print(f"Failures: {breaker.failures}")


# Run the example
await circuit_breaker_demo()

### Circuit Breaker States

```
CLOSED (normal operation)
    ↓ (failures >= threshold)
OPEN (reject all calls)
    ↓ (timeout expires)
HALF-OPEN (test recovery)
    ↓ (success)        ↓ (failure)
  CLOSED             OPEN
```

**Benefits**:
- Prevents wasted requests to failing services
- Allows service time to recover
- Fails fast during outages

## 4. Fallback Chain

Graceful degradation by trying multiple providers until one succeeds.

In [ ]:
async def fallback_chain():
    """Try multiple providers until one succeeds"""
    session = Session()

    # Register multiple models as fallbacks
    providers = [
        iModel(provider="openai", model="gpt-4o-mini", temperature=0, name="primary"),
        iModel(
            provider="anthropic",
            endpoint="messages",
            model="claude-3-5-haiku-20241022",
            temperature=0,
            name="fallback1",
        ),
        iModel(provider="gemini", model="gemini-2.0-flash-exp", temperature=0, name="fallback2"),
    ]

    for model in providers:
        session.services.register(model)

    branch = session.create_branch(name="fallback")
    question = "What is machine learning in one sentence?"

    # Try each provider in order
    for i, model in enumerate(providers, 1):
        try:
            print(f"Attempt {i}: Trying {model.name}...")

            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": question,
                    "imodel": model.name,
                },
            )

            print(f"✓ Success with {model.name}")
            print(f"Result: {result}\n")
            return result

        except Exception as e:
            print(f"✗ {model.name} failed: {type(e).__name__}")
            if i == len(providers):
                print("All providers failed!")
                raise
            print("Falling back to next provider...\n")


# Run the example
await fallback_chain()

### Fallback Strategy

**Order by**:
1. **Primary**: Best quality/speed
2. **Secondary**: Good balance
3. **Tertiary**: Always available (or local model)

**Benefits**:
- High availability across provider outages
- Cost optimization (try cheaper models first if appropriate)
- Provider diversity reduces single points of failure

## 5. Validation Error Recovery

When structured outputs fail validation, retry with corrective instructions.

In [ ]:
class ExtractedData(BaseModel):
    """Structured data with validation"""

    name: str = Field(..., min_length=1)
    age: int = Field(..., ge=0, le=150)
    email: str = Field(..., pattern=r"^[\w\.-]+@[\w\.-]+\.\w+$")


async def validate_and_recover():
    """Validate outputs and retry with corrections"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)

    branch = session.create_branch(name="validation")

    max_attempts = 3
    for attempt in range(1, max_attempts + 1):
        print(f"\nAttempt {attempt}:")

        try:
            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": (
                        "Extract person info: 'John Doe, 32 years old, email: john.doe@example.com'"
                    ),
                    "imodel": model.name,
                    "response_model": ExtractedData,
                    "return_as": "model",
                },
            )

            print("✓ Validation passed!")
            print(f"  Name: {result.name}")
            print(f"  Age: {result.age}")
            print(f"  Email: {result.email}")
            return result

        except ValidationError as e:
            print(f"✗ Validation failed: {e}")

            if attempt < max_attempts:
                # Add corrective instruction
                error_details = str(e)
                print("  Retrying with correction...")
            else:
                print("Max attempts reached!")
                raise


# Run the example
await validate_and_recover()

### Validation Recovery Pattern

1. **Try extraction** with response_model
2. **Catch ValidationError** with specific field failures
3. **Send corrective instruction** with error details
4. **Retry** with updated context
5. **Fail after max attempts** to prevent infinite loops

**Key**: Use Pydantic's detailed error messages to guide the model's correction.

## 6. Timeout Handling

Set time boundaries to prevent indefinite waits.

In [ ]:
async def timeout_handling():
    """Handle operation timeouts using lionpride's fail_after"""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="timeout")

    try:
        # Use lionpride's fail_after for deadline-aware timeout
        with fail_after(30.0):  # 30 second timeout
            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": "Write a brief summary about the history of computing",
                    "imodel": model.name,
                    "max_tokens": 200,
                },
            )
        print(f"✓ Completed within timeout: {result}")

    except TimeoutError:
        print("✗ Operation timed out after 30 seconds")

        # Option: Retry with shorter response
        print("Retrying with shorter max_tokens...")
        with fail_after(10.0):
            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": "Write one sentence about computing history",
                    "imodel": model.name,
                    "max_tokens": 50,
                },
            )
        print(f"✓ Completed: {result}")


# Run the example
await timeout_handling()

### Timeout Primitives (lionpride.libs.concurrency)

**fail_after(seconds)**: Raises TimeoutError if deadline exceeded
**move_on_after(seconds)**: Silently cancels if deadline exceeded (no exception)

**Recommended timeouts**:
- **Short responses**: 5-10 seconds
- **Medium responses**: 15-30 seconds
- **Long responses**: 30-60 seconds
- **Batch operations**: 60-300 seconds

**Always set timeouts** to prevent hanging operations and resource leaks.

## 7. Rate Limit Handling

Built-in rate limiting prevents hitting provider quotas.

In [ ]:
async def rate_limit_handling():
    """Handle rate limits gracefully"""
    session = Session()

    # Use built-in rate limiting
    model = iModel(
        provider="openai",
        model="gpt-4o-mini",
        temperature=0,
        limit_requests=10,  # Max 10 requests per minute
        limit_tokens=10000,  # Max 10k tokens per minute
        capacity_refresh_time=60,
    )
    session.services.register(model)

    # Send multiple requests
    num_requests = 5
    print(f"Sending {num_requests} requests with rate limit of 10/minute...\n")

    results = []
    for i in range(num_requests):
        try:
            branch = session.create_branch(name=f"request-{i}")
            result = await communicate(
                session=session,
                branch=branch,
                parameters={
                    "instruction": f"Question {i + 1}: What is {i + 1} squared?",
                    "imodel": model.name,
                },
            )
            print(f"Request {i + 1}: ✓ {result}")
            results.append(result)
        except Exception as e:
            print(f"Request {i + 1}: ✗ {e}")

    print(f"\nCompleted {len(results)}/{num_requests} requests")


# Run the example
await rate_limit_handling()

### Rate Limit Configuration

**Parameters**:
- `limit_requests`: Max requests per time window
- `limit_tokens`: Max tokens per time window
- `capacity_refresh_time`: Time window in seconds

**Automatic throttling** prevents 429 errors and quota violations.

## Production-Ready Patterns

### Best Practices

#### 1. Fail Fast for Configuration Errors

```python
# Validate early
if not session.services.has(model_name):
    raise ValueError(f"Model {model_name} not registered")

# Don't retry configuration errors
```

#### 2. Retry Transient Errors

```python
from lionpride.libs.concurrency import retry

# Network errors, rate limits, timeouts
result = await retry(
    lambda: communicate(session, branch, params),
    attempts=3,
    retry_on=(RuntimeError, ConnectionError, TimeoutError),
)
```

#### 3. Log Failures

```python
import logging

logger = logging.getLogger(__name__)

try:
    result = await communicate(...)
except Exception as e:
    logger.error(f"Communication failed: {e}", exc_info=True)
    raise
```

#### 4. Always Use Timeouts

```python
from lionpride.libs.concurrency import fail_after

# Always set reasonable timeouts
with fail_after(30.0):  # 30 seconds
    result = await long_operation()
```

#### 5. Graceful Degradation

```python
# Provide fallback responses
try:
    result = await ai_analysis()
except Exception:
    result = "Analysis unavailable. Please try again later."
```

## Common Pitfalls

### 1. Catching Too Broad

```python
# ❌ Wrong - masks real issues
try:
    result = await communicate(...)
except:  # Catches everything!
    pass

# ✅ Right - specific exceptions
try:
    result = await communicate(...)
except (ValueError, RuntimeError) as e:
    handle_error(e)
```

### 2. Infinite Retries

```python
# ❌ Wrong - no limit
while True:
    try:
        return await communicate(...)
    except:
        continue  # Forever!

# ✅ Right - max attempts
for attempt in range(max_retries):
    try:
        return await communicate(...)
    except:
        if attempt == max_retries - 1:
            raise
```

### 3. Not Handling Async Errors

```python
# ❌ Wrong - async error not awaited
try:
    task = communicate(...)  # Not awaited!
except Exception:
    pass  # Won't catch

# ✅ Right - await first
try:
    result = await communicate(...)
except Exception:
    handle_error()
```

## Summary

### Key Takeaways

1. **Handle specific exceptions** - Don't catch everything
2. **Retry transient failures** - Use exponential backoff
3. **Set timeouts** - Prevent hanging operations
4. **Use circuit breakers** - Prevent cascading failures
5. **Implement fallbacks** - Graceful degradation across providers
6. **Validate outputs** - Retry with corrections
7. **Rate limit proactively** - Prevent quota violations
8. **Log failures** - Aid debugging and monitoring

### Production Checklist

- [ ] Specific exception handling
- [ ] Retry logic with max attempts
- [ ] Exponential backoff
- [ ] Circuit breakers for critical paths
- [ ] Fallback providers configured
- [ ] Timeouts set on all async operations
- [ ] Rate limits configured
- [ ] Comprehensive logging
- [ ] Graceful degradation
- [ ] Monitoring and alerts

### Next Steps

- **Advanced patterns**: See `09_advanced_operations.ipynb`
- **Testing**: See `10_testing.ipynb`
- **Production deployment**: See documentation

Build resilient, production-ready AI applications! 🚀